In [1]:
# Import libraries
import numpy as np 
import matplotlib.pyplot as plt
import os
import torch
import torch.nn as nn
from torch.autograd import Variable
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset

In [2]:
# Prepare Dataset
# load data
filename_0 = 'output_batch_0.npz'
filename_1 = 'output_batch_1.npz'
filename_2 = 'output_batch_2.npz'
filename_3 = 'output_batch_3.npz'
filename_4 = 'output_batch_4.npz'
filename_5 = 'output_batch_5.npz'
filename_6 = 'output_batch_6.npz'
filename_7 = 'output_batch_7.npz'
filename_8 = 'output_batch_8.npz'
filename_9 = 'output_batch_9.npz'
filename_10 = 'output_batch_10.npz'
filename_11 = 'output_batch_11.npz'
folder_path = 'Measurements'
current_directory = 'c:/Users/20201558/Desktop/Work related stuff/Master/Year 4/Quartile 3/Interdisciplinary Team Project [5ARIP10]/Code/VBSS'
file_path_0 = os.path.join(current_directory,folder_path, filename_0)
file_path_1 = os.path.join(current_directory,folder_path, filename_1)
file_path_2 = os.path.join(current_directory,folder_path, filename_2)
file_path_3 = os.path.join(current_directory,folder_path, filename_3)
file_path_4 = os.path.join(current_directory,folder_path, filename_4)
file_path_5 = os.path.join(current_directory,folder_path, filename_5)
file_path_6 = os.path.join(current_directory,folder_path, filename_6)
file_path_7 = os.path.join(current_directory,folder_path, filename_7)
file_path_8 = os.path.join(current_directory,folder_path, filename_8)
file_path_9 = os.path.join(current_directory,folder_path, filename_9)
file_path_10 = os.path.join(current_directory,folder_path, filename_10)
file_path_11 = os.path.join(current_directory,folder_path, filename_11)
data_0 = np.load(file_path_0)
data_1 = np.load(file_path_1)
data_2 = np.load(file_path_2)
data_3 = np.load(file_path_3)
data_4 = np.load(file_path_4)
data_5 = np.load(file_path_5)
data_6 = np.load(file_path_6)
data_7 = np.load(file_path_7)
data_8 = np.load(file_path_8)
data_9 = np.load(file_path_9)
data_10 = np.load(file_path_10)
#data_11 = np.load(file_path_11)
data_frames = [data_0['frames'], data_1['frames'], data_2['frames'], data_3['frames'], data_4['frames'],
               data_5['frames'], data_6['frames'], data_7['frames'], data_8['frames'], data_9['frames'], data_10['frames']]

data_forces = [data_0['forces'], data_1['forces'], data_2['forces'], data_3['forces'], data_4['forces'],
               data_5['forces'], data_6['forces'], data_7['forces'], data_8['forces'], data_9['forces'], data_10['forces']]

combined_frames = np.concatenate(data_frames, axis=0)
combined_forces = np.concatenate(data_forces, axis=0)

In [3]:
# train test split. Size of train data is 80% and size of test data is 20%. 
# features_train, features_test, targets_train, targets_test = train_test_split(combined_frames,
#                                                                              combined_forces[:,2],
#                                                                              test_size = (1/11),
#                                                                              random_state = 42) 
features_train = combined_frames
targets_train = combined_forces[:,2]
# create feature and targets tensor for train set. As you remember we need variable to accumulate gradients. Therefore first we create tensor, then we will create variable
featuresTrain = torch.from_numpy(features_train)
targetsTrain = torch.from_numpy(targets_train).type(torch.LongTensor) # data type is long

# create feature and targets tensor for test set. 
# featuresTest = torch.from_numpy(features_test)
# targetsTest = torch.from_numpy(targets_test).type(torch.LongTensor) # data type is long

In [4]:
# Create RNN Model
class RNNModel(nn.Module):
    def __init__(self, input_channels, hidden_dim, layer_dim, output_dim):
        super(RNNModel, self).__init__()
        
        # Convolutional layers
        self.conv1 = nn.Conv2d(input_channels, 12, kernel_size=5, stride=1, padding=2)
        self.pool = nn.MaxPool2d(kernel_size=4, stride=4)
        #self.conv2 = nn.Conv2d(12, 36, kernel_size=5, stride=1, padding=2) # If needed
        
        # RNN
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim
        self.rnn = nn.RNN(12 * 67 * 120, hidden_dim, layer_dim, batch_first=True, nonlinearity='relu')
        
        # Readout layer
        self.fc = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x):
        # Convolutional layers
        x = self.pool(nn.ELU()(self.conv1(x)))
        #x = self.pool(nn.ELU()(self.conv2(x))) # If needed
        
        # Reshape for RNN
        x = torch.reshape(x, (-1, 1, 12 * 67 * 120))  # Reshape to (batch_size, seq_len, input_size)
        print("Size of x:", x.size())  # Print size of x
        
        # RNN
        h0 = Variable(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim))
        #print("Size of h0:", h0.size())  # Print size of h0 - Debugging - Obsolete
        out, hn = self.rnn(x, h0)
        
        # Output layer
        out = self.fc(out[:, -1, :]) 
        return out


In [5]:
# batch_size, epoch and iteration
batch_size = 1000
num_epochs = 2

# Pytorch train and test sets
train = TensorDataset(featuresTrain,targetsTrain)
# test = TensorDataset(featuresTest,targetsTest)

# data loader
train_loader = DataLoader(train, batch_size = batch_size, shuffle = False)
# test_loader = DataLoader(test, batch_size = batch_size, shuffle = False)
    
# Create RNN
input_channels = 3  # RGB channels
hidden_dim = 100  # hidden layer dimension
layer_dim = 1     # number of hidden layers
output_dim = 1   # output dimension

model = RNNModel(input_channels, hidden_dim, layer_dim, output_dim)

# Define your loss function
error = nn.MSELoss()

# Define your optimizer
learning_rate = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [6]:
# Training
seq_dim = 50 # Consider the whole batch to be temporally correlated
loss_list = []
iteration_list = []
accuracy_list = []
count = 0
for epoch in range(num_epochs):
    print('~~~BEGINNING OF DATASET~~~')
    for i, (images, labels) in enumerate(train_loader):
        images = images.float()
        # print(images.shape)  # Add this line to check the shape of images - Debugging purposes
        images = images.permute(0, 3, 1, 2)
            
        # Clear gradients
        optimizer.zero_grad()
        
        # Forward propagation
        outputs = model(images)
        outputs = torch.squeeze(outputs)
        
        # Calculate loss
        loss = error(outputs, labels.float())
        
        # Backpropagation
        loss.backward()
        
        # Update parameters
        optimizer.step()
        
        count += 1
            
        # Store loss and iteration
        loss_list.append(loss.data)
        # Print Loss
        if count % 1 == 0: # for now print for every iteration
            print('Iteration: {}  Loss: {}'.format(count, loss.data.item()))

~~~BEGINNING OF DATASET~~~
Size of x: torch.Size([1000, 1, 96480])
Iteration: 1  Loss: 421208.8125
Size of x: torch.Size([1000, 1, 96480])
Iteration: 2  Loss: 49158796.0
Size of x: torch.Size([1000, 1, 96480])
Iteration: 3  Loss: 6980169.0
Size of x: torch.Size([1000, 1, 96480])
Iteration: 4  Loss: 201368.734375
Size of x: torch.Size([1000, 1, 96480])
Iteration: 5  Loss: 482554.9375
Size of x: torch.Size([1000, 1, 96480])
Iteration: 6  Loss: 231398.203125
Size of x: torch.Size([1000, 1, 96480])
Iteration: 7  Loss: 52016.48046875
Size of x: torch.Size([1000, 1, 96480])
Iteration: 8  Loss: 90032.65625
Size of x: torch.Size([1000, 1, 96480])
Iteration: 9  Loss: 25411.142578125
Size of x: torch.Size([1000, 1, 96480])
Iteration: 10  Loss: 30439.662109375
Size of x: torch.Size([1000, 1, 96480])
Iteration: 11  Loss: 15626.3466796875
~~~BEGINNING OF DATASET~~~
Size of x: torch.Size([1000, 1, 96480])
Iteration: 12  Loss: 1724.7554931640625
Size of x: torch.Size([1000, 1, 96480])
Iteration: 13  

In [7]:
print(loss_list)

[tensor(421208.8125), tensor(49158796.), tensor(6980169.), tensor(201368.7344), tensor(482554.9375), tensor(231398.2031), tensor(52016.4805), tensor(90032.6562), tensor(25411.1426), tensor(30439.6621), tensor(15626.3467), tensor(1724.7555), tensor(2254.0117), tensor(10604.4189), tensor(12004.6660), tensor(6137.0552), tensor(847.7997), tensor(510.7110), tensor(3534.5725), tensor(5471.4800), tensor(4011.5505), tensor(2414.4070)]


In [8]:
print(outputs) # Small sanity check

tensor([-48.2465, -51.8231, -49.5471, -52.0802, -48.4703, -49.5593, -49.8716,
        -46.6503, -43.4998, -45.6609, -48.6962, -49.9108, -48.3651, -51.0996,
        -51.7064, -51.6190, -54.2974, -51.8446, -55.0375, -56.5656, -52.4609,
        -50.7762, -54.7804, -56.6121, -50.5061, -54.9269, -55.8640, -52.9303,
        -53.9789, -55.3998, -48.7527, -55.5217, -54.0393, -57.2656, -53.8654,
        -58.3008, -60.1141, -57.1390, -61.8564, -56.9197, -58.4042, -58.9843,
        -61.0719, -62.6067, -59.5601, -55.9377, -54.3970, -56.7759, -53.9485,
        -51.6389, -53.9924, -54.8955, -51.4584, -50.4302, -52.6790, -53.6582,
        -54.9213, -54.4041, -46.4554, -47.5317, -45.7191, -47.9839, -47.3402,
        -47.9602, -44.3272, -44.5654, -44.0790, -44.5206, -44.9143, -43.8154,
        -45.2762, -41.7671, -43.1913, -44.5996, -42.7648, -45.2048, -46.9114,
        -43.7299, -44.2434, -44.8311, -45.7046, -49.3543, -44.3602, -48.2489,
        -46.5252, -48.1691, -46.6436, -47.3927, -47.7946, -48.33

In [9]:
print(labels.float())

tensor([-2., -2., -2., -3., -3., -3., -3., -3., -3., -3., -3., -2., -2., -2.,
        -2., -2., -1., -1., -1., -1., -1., -1.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0., -1., -1., -1., -1., -1., -1., -2.,
        -2., -2., -2., -2., -3., -3., -3., -3., -3., -3., -3., -2., -2., -2.,
        -2., -2., -2., -1., -1., -1., -1., -1., -1., -1.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0., -1., -1., -1., -1., -1., -1., -1., -1., -2., -2., -2., -2.,
        -2., -2., -2., -3., -3., -3., -3., -3., -3., -3., -3., -3., -3., -3.,
        -2., -2., -2., -2., -1., -1., -1., -1., -1., -1., -1.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0., -1., -1., -1., -1., -1., -1., -2., -2., -2., -

Progress Summary: Run over the whole dataset (without batch 11) 2 times completed successfully. The model does learn from the dataset. Interesting shift (50 RMSE) in outputs highlighted in the cells above. It could be that simply more time is needed for the model to approach the (-5,0) Newton range. Data is not shuffled (to preserve time-related aspects), overfitting likely to occur in further runs. We can use batch_11.npz as test set maybe.
